# Processo de Extração de Leads (SEMANAL!!!)

Para facilitar o processo é necessário executar esse script com as bases extraidas do hubspot.

Aqui você vai conseguir juntar todas as planilhas separando os dados por appeal e filtrar os telefones.

Para iniciar é necessário executar a celula abaixo e selecionar os arquivos da extração e o template.


In [0]:
# Importando arquivos de extração do HubSpot e template

# Aqui você vai importar multiplos arquivos que ficarão disponiveis para visualização na aba files a esquerda ( <- )
# Será necessário fazer o upload de todos os arquivos para que o script funcione

# Você pode reexecutar essa célula várias vezes caso ocorra algum problema

from google.colab import files

uploaded = files.upload()

# \/ \/ \/ Selecione todos os arquivos na interação abaixo! \/ \/ \/

In [0]:
import pandas as pd
import numpy as np
import glob
import re
from openpyxl import load_workbook
import warnings


warnings.filterwarnings('ignore')
# Importando módulos necessários

In [0]:
# Criando Data Frame principal, que irá abrigar todas as extrações.

all_data = pd.DataFrame()

# Tratamento das tabelas

A partir daqui vamos juntar todos os arquivos da extração em uma unica tabela, que será filtrada pela REGEX, uma fórmula que identifica padrões de telefone que estão incorretos ou não existem.

In [0]:
# Busca todos os arquivos EXCEL no diretório local do colab.​

for files in glob.glob('/content/hubspot*.xlsx'):
   appealCampaign = re.search(".*", files) 
   df = pd.read_excel(files).assign(Appeal=appealCampaign.group())
   all_data = all_data.append(df, ignore_index=True)#.copy()
   all_data.head()


# Essa célula concatena todos os arquivos de extração em um único DataFrame (tabela pandas) 
# e adiciona uma coluna com o nome do arquivo, para identificar qual a fonte do Lead

In [0]:
# Célula responsavel por organizar o writer que vai salvar o arquivo de acordo com as abas do template.

book = load_workbook('/content/Template.xlsx')
writer = pd.ExcelWriter('/content/Template.xlsx', engine='openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

In [0]:
all_data.replace(to_replace ='''
"^(\+55|55|055|\s\+55|\s55|\s055){0,1}?(\s|-|\.|\+|\_)?(0{0,1}\s{0,1})?
([1-9][0-9]|\([1-9][0-9]\))(\s|-|\.|\+|\_|\(|\)){0,2}?((?!9999)\d{4,5})
{0,1}?(\s|-|\.|\+|\_)?((?!0000)?\d{4,5})$"''', value=r"\4-\6\8", regex=True, inplace=True)

In [0]:
all_data.filter(regex="^(\+55|55|055|\s\+55|\s55|\s055){0,1}?(\s|-|\.|\+|\_)?(0{0,1}\s{0,1})?([1-9][0-9]|\([1-9][0-9]\))(\s|-|\.|\+|\_){0,2}?(\d{4,5}){0,1}?(\s|-|\.|\+|\_)?((?!0000)?\d{4,5})$", axis=1)

In [0]:
all_data.insert(6, column='validacao de len', value = 'none')

In [0]:
# Loop adicional de filtros de DD e length dos telefones.

for index in range(len(all_data)):
   ad = all_data['Phone Mask'][index]
   try:
      if len(ad) == 12 and ad[3] != '9':
         all_data['validacao de len'][index] = 'Não começa com 9'
      elif ad[1] == '0':
         all_data['validacao de len'][index] = 'DDD inválido'
      elif ad[0] == '0':
         all_data['validacao de len'][index] = 'DDD inválido'
      elif len(ad) < 11:
         all_data['validacao de len'][index] = 'Curto demais'
      elif len(ad) > 12:
         all_data['validacao de len'][index] = 'Longo demais'
      elif len(ad) == 11 and (ad[3] == '6' or ad[3] == '7' or ad[3] == '8' or ad[3] == '9'):
         all_data['validacao de len'][index] = 'Não tem o nono dígito'
      elif len(ad) > 12:
         all_data['validacao de len'][index] = 'Longo demais'
      elif len(ad) == 'NaN':
         all_data['validacao de len'][index] = 'Telefone não informado'
            
   except:
      all_data['validacao de len'][index] = 'Não conseguiu identificar o número'

# Finalização

Execute a ultima célula abaixo para concluir o processo, o arquivo será salvo na pasta ao lado (<-) do seu diretório do Colab.

Lembrando que é importante conferir os processos de execução e se certificar que os filtros estão funcionando de forma adequada.


Qualquer dúvida contate a chatuba através do e-mail:
renato.fagaraz@greenpeace.org

In [0]:
all_data.to_excel(writer, sheet_name='Base Original')
writer.save()